In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches
#import matplotlib.font_manager as fm
import matplotlib
import autotime
%matplotlib inline
%load_ext autotime
#import seaborn as sns
import statsmodels as stm
import statsmodels.formula.api as sm
from sklearn.feature_selection import VarianceThreshold
from sklearn import metrics
from sklearn.linear_model import LinearRegression as LinR
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV as RSCV
# from dask_searchcv import RandomizedSearchCV as DRSCV
# from sklearn.pipeline import Pipeline
from sklearn.metrics.scorer import make_scorer
# from sklearn.preprocessing import MinMaxScaler as MMS
from scipy import stats
from scipy.stats import randint
from sklearn.externals import joblib
import os

#from sklearn.tree import export_graphviz
#from IPython.display import Image

### Before running this program, create folder named SSID_XXXX_model_GBR_no_outliers_cut_pkls to store the outputted model pickle files

#### Following is prepare to get the SSID array of routes

In [3]:
# Read googletransit data in

# service_id to its service day
calendar = pd.read_csv('./googletransitdbfeedp20121129-1547/calendar.txt')

# route_id to its route_short_name, route_long_name
routes = pd.read_csv('./googletransitdbfeedp20121129-1547/routes.txt')

# stop_id to its stop_name and lat/lon
# (we only need "8220DB000002" => "0002" as stop_id)
stops = pd.read_csv('./googletransitdbfeedp20121129-1547/stops.txt')

# trip_id to its service_id, route_id, trip_headsign 
# (we only need "4777.2.0-1-y12-1.1.O"=> "0-1-y12-1.1.O"(shape_id) as trip_id)
trips = pd.read_csv('./googletransitdbfeedp20121129-1547/trips.txt')

# trip_id to its stop_sequence, stop_id and time_table (no route_id in this file)
# (we only need "4777.2.0-1-y12-1.1.O"=> "0-1-y12-1.1.O" as trip_id and "0-1-y12-1" as route_id)
stop_times = pd.read_csv('./googletransitdbfeedp20121129-1547/stop_times.txt')

time: 1.16 s


In [4]:
# stops
# 0805 revised: clean duplicate.
stops = stops[stops.stop_id.str.find('VIR') == -1]
stops = stops[~stops.stop_id.isin(['8220PB003854', '8250DB00661'])]
stops.set_value(index=stops[stops.stop_id == '8220DB007270'].index, col='stop_id', value='8220DB007220')
stops.set_value(index=stops[stops.stop_id == '8250DB002561'].index, col='stop_id', value='8250DB007550')

# stop_times

stop_times = stop_times[stop_times.stop_id.str.find('VIR') == -1]
stop_times = stop_times[~stop_times.stop_id.isin(['8220PB003854', '8250DB00661'])]
stop_times.set_value(index=stop_times[stop_times.stop_id == '8220DB007270'].index, col='stop_id', value='8220DB007220')
stop_times.set_value(index=stop_times[stop_times.stop_id == '8250DB002561'].index, col='stop_id', value='8250DB007550')

stop_times = pd.merge(stop_times, stops, on='stop_id', how='inner')
stop_times.drop(['stop_name', 'stop_lat', 'stop_lon'], axis=1, inplace=True)
stops['stop_id'] = stops['stop_id'].str[-4:]
stop_times['trip_id'] = stop_times['trip_id'].apply(lambda x: x[x.index('.0')+1:])
stop_times['stop_id'] = stop_times['stop_id'].str[-4:]
stop_times['route_id'] = stop_times['trip_id'].apply(lambda x: x[: x.index('.')])

# Get route name to its corresponding SSID list
gb = stop_times.groupby(['trip_id','stop_sequence','stop_id'])
temp = gb.count()
trip_stops = temp.reset_index(['trip_id','stop_sequence','stop_id'])
trip_stops = trip_stops.loc[:, ['trip_id','stop_sequence','stop_id']]
trip_stops.sort_values(['trip_id','stop_sequence'], inplace=True)
trip_stops = trip_stops.drop_duplicates()

time: 2.72 s


In [5]:
# Get the SSID array for each routes
trip_id_unique = trip_stops.trip_id.unique()

target = []
for key in trip_id_unique:
    
    sub_group_idx = trip_stops[trip_stops['trip_id'] == key].index

    passby_list = []    
    passby_list.append(trip_stops['stop_id'].iloc[sub_group_idx[0]])
    target.append(tuple(passby_list))
    
    count = 0
    for j in sub_group_idx:
        if count == 0:
            count += 1
            continue
        passby_list.append(trip_stops['stop_id'].iloc[j])
        target.append(tuple(passby_list))

time: 1.33 s


In [6]:
def get_ssid_array(x):
    ssid_array = []
    for i in range(len(x)-1):
        ssid_array.append(x[i]+x[i+1])
    return ssid_array

time: 1 ms


In [7]:
target_1 = [list(x) for x in target]
trip_stops['pass_stop_id'] = np.array(target_1)
trip_stops['pass_ssid'] = trip_stops['pass_stop_id'].apply(lambda x: get_ssid_array(x))

time: 205 ms


In [8]:
trip_stops['route'] = trip_stops['trip_id'].apply(lambda x: x[x.index('-')+1:])
trip_stops['route'] = trip_stops['route'].apply(lambda x: x[:x.index('-')])
trip_stops['route'] = trip_stops['route'].str.zfill(4)
trip_ssids = trip_stops[trip_stops.groupby(['route'])['stop_sequence'].transform('max') == trip_stops['stop_sequence']]
trip_stops.head(3)

,trip_id,stop_sequence,stop_id,pass_stop_id,pass_ssid,route
0,0-1-y12-1.1.O,1,0226,[0226],[],0001
1,0-1-y12-1.1.O,2,0228,"[0226, 0228]",[02260228],0001
2,0-1-y12-1.1.O,3,0229,"[0226, 0228, 0229]","[02260228, 02280229]",0001


time: 45 ms


#### End preparation

In [16]:
# Put the training procedure here
def train_model(SSIDno, res):
    

    # ------------------------------#
    # Prepare dataframe for model   #
    # ------------------------------#
    
    # create dataframe for SSIDno

    res['SSID'] = res['SSID'].astype('category')
    #ssid_df = res[res.SSID == SSIDno]
    ssid_df = res
    
    ssid_df.reset_index(inplace=True)
    ssid_df = ssid_df.drop('index', axis=1)
    
    # create JPID_Count variable
    JPID_Count = ssid_df.JourneyPatternID.unique().shape[0]
    Row_Count = ssid_df.shape[0]
    
    # returning if less than 5 entries
    if Row_Count < 5:
        return
    
    res.SSID = res.SSID.apply(lambda x: str(int(x)).zfill(8))
    r = ssid_df.loc[0, 'SSID'] 
    
    # add leading zeroes to JourneyPatternID

    ssid_df.JourneyPatternID = ssid_df.JourneyPatternID.astype('object')
    ssid_df.JourneyPatternID = ssid_df.JourneyPatternID.apply(lambda x: str(x).zfill(8))
    
    # adding Xbuses feature - boolean feature indicating whether or not the JourneyPatternID represents an express (X) bus

    ssid_df['XBuses'] = ssid_df[ssid_df["JourneyPatternID"].str.find("X") > 0].sum(axis=1) > 0
    ssid_df["XBuses"].fillna(False, inplace=True)
    ssid_df['XBuses'] = ssid_df['XBuses'].astype('int')
    
    # adding JPID_length feature - represents the total number of stops traversed by this JourneyPatternID along its entire route

    JPIDL = pd.read_csv('JPID_Length.csv')
    JPIDL = JPIDL.drop('Unnamed: 0', axis=1)
    ssid_df = pd.merge(left=ssid_df ,right=JPIDL, how='left', left_on='JourneyPatternID', right_on='JourneyPatternID')
    ssid_df = ssid_df.dropna()
    
    # removing non-holiday-period weekends from SchoolHoliday feature to avoid multi-collinearity issues

    ssid_df['SchoolHoliday'] = ssid_df['SchoolHoliday'].astype('int')
    ssid_df['SchoolHoliday'] = np.where(ssid_df['TimeFrame'].isin(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04', '2013-01-05', '2013-01-06']), 1, 0)
    
    # selecting only required columns

    ssid_df = ssid_df[['TravelTime', 'Rain', 'WindSpeed', 'JPID_length', 'XBuses', 'SchoolHoliday', 'Day', 'HourFrame']]
    
    # assigning appropriate datatypes where necessary

    ssid_df['Day'] = ssid_df['Day'].astype('category')
    ssid_df['HourFrame'] = ssid_df['HourFrame'].astype('category')
    ssid_df['JPID_length'] = ssid_df['JPID_length'].astype('int')
    
    # removing any constant features
    # code from: https://stackoverflow.com/questions/20209600/panda-dataframe-remove-constant-column

    ssid_df = ssid_df.loc[:,ssid_df.apply(pd.Series.nunique) != 1]
    
    # Create TT mean/median value variables, to use in calculating mean/median absolute percentage accuracy scores

    ssid_df_mean = ssid_df.TravelTime.mean()
    ssid_df_median  = ssid_df.TravelTime.median()
    
    # ------------------------------#
    #         Model Training        #
    # ------------------------------#
    
    # create dummy variables from HourFrame and Day using get_dummies
    # check they exist first (have been dropped if constant)

    if 'Day' in ssid_df.columns:
        Day_dummies = pd.get_dummies(ssid_df.Day, prefix='Day', drop_first=True)
        ssid_df = pd.concat([ssid_df, Day_dummies], axis=1)
        ssid_df = ssid_df.drop(['Day'], axis=1)
        
    if 'HourFrame' in ssid_df.columns:
        HF_dummies = pd.get_dummies(ssid_df.HourFrame, prefix='HF', drop_first=True)
        ssid_df = pd.concat([ssid_df, HF_dummies], axis=1)
        ssid_df = ssid_df.drop(['HourFrame'], axis=1)
    
    # prepare a list containing all remaining features bar the target
    pred_features = list(ssid_df)
    pred_features.remove('TravelTime')
    
    # prepare target/predictive feature variables for use in scikit-learn modelling

    X = ssid_df[pred_features]
    y = ssid_df['TravelTime']
    
    # creating blank results dataframe
    
    results = pd.DataFrame(columns=['SSID', 'R-squared', 'MAPE', 'MdAPE', 'Row_Count', 'JPID_Count'])
    
    # ------------------------------------------------------------------------------------------#
    ### Gradient Boosting Regression model - training on full datasets with chosen parameters ###
    # ------------------------------------------------------------------------------------------#
    
    gbr = GBR(loss='huber')
    gbr.fit(X, y)
    
    # producing metric variables for CSV

    gbr_pred = gbr.predict(X)
    gbr_rsq = metrics.r2_score(y, gbr_pred)

    gbr_mae = metrics.mean_absolute_error(y, gbr_pred)
    gbr_mape = (gbr_mae/ssid_df_mean)*100

    gbr_mdae = metrics.median_absolute_error(y, gbr_pred)
    gbr_mdape = (gbr_mdae/ssid_df_median)*100
    
    temp = pd.DataFrame.from_dict({'SSID':SSIDno, 'R-squared':gbr_rsq,'MAPE':gbr_mape,'MdAPE':gbr_mdape, 'Row_Count':Row_Count, 'JPID_Count':JPID_Count}, orient='index')
    temp = temp.T
    results = pd.concat([results, temp], axis=0)
    
    fo = 'model_results_GBR_no_outliers_cut.csv'
    if not os.path.isfile(fo):
        results.to_csv(fo,index=False)
    else:
        results.to_csv(fo,index=False, header=False, mode='a')
    
    ## return the model
    return gbr

time: 198 ms


In [17]:
# Store trained model to pickle files
def store_ssid_as_pickle():
    
    #ssid_array = trip_ssids[trip_ssids.route == route]['pass_ssid'].values[0]
    allSSID = pd.read_csv('allSSID.csv')
    ssid_array = []
    for s in allSSID['SSID']:
        ssid_array.append(str(s).zfill(8))
    
    for ssid in ssid_array:
        print(ssid)
        
        # Open ssid
        blank = pd.read_csv('./Route_XXXX_travel_time_csvs/Blank_Route_travel_time.csv')
        blank.drop(0, axis=0, inplace=True)
        fn = './SSID_XXXX_travel_time_csvs/travel_time_' + ssid + '.csv'
        if not os.path.isfile(fn):
            continue
        temp = pd.read_csv(fn)
        res = pd.concat([blank, temp], axis=0) 
        res['TravelTime'] = res['TravelTime'].astype('float64')
        
        # Call model function
        ssid_model = train_model(ssid, res)
        
        # output pkl file by joblib
        pickle_name = './SSID_XXXX_model_GBR_no_outliers_cut_pkls/' + ssid + '.pkl'
        joblib.dump(ssid_model, pickle_name)
        
    return

time: 11 ms


In [18]:
# produce all pickle models
store_ssid_as_pickle()

47694770
35853587
14781479
29192920
18277382
00400146
38097174
72954199
22127165
33963397
37813782
60146242
04020405
22492250
13893355
29427065
43313641
07800781
34713472
41564169
38963897
22462247
70564731
21343378
42664307
25392540
16614523
24502328
01410142
02420243
70626075
01360137
35622899
47477374
06280629
15840983
06860688
47454989
50733679
31713173
70797080
51554806
26411948
00900091
24272430
49274928
26162617
20373046
25480946
12504698
20712072
74532002
61446145
11221123
22052687
03340406
26362383
74614227
61820322
21402141
24152416
06750621
16661807
16061607
12861288
74587425
21492150
71723775
11000100
16481649
35253527
31873188
06490650
29257442
41094110
07990800
08120813
16951696
49074908
43364337
36567194
51420448
61656157
42924293
18372338
36573658
41824151
35543555
29142989
07230724
05270528
22385056
43361051
61976198
04293032
10061007
41084109
15151499
25797395
50754915
05850586
61244004
10051788
38023805
35644477
18382338
61856186
07450600
21684536
31823183
71227121
4